In [1]:
import numpy as np
import pandas as pd
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
# from pickle import NONE
# IMAGE_NUM = 7129
# path =  "/content/drive/MyDrive/image_to_str/sample_data/"
# train_list = []
# for i in range(1001):
#       img = cv2.imread(path + str(i)+".jpg",cv2.IMREAD_GRAYSCALE)
#       train_list.append(img)

# test_list = []

# for i in range(5600,IMAGE_NUM):
#     img = cv2.imread(path + str(i)+".jpg",cv2.IMREAD_GRAYSCALE)
#     test_list.append(img)






In [6]:
x_transform = A.Compose(
    [
     A.Resize(150,150),
     A.HorizontalFlip(p=0.5),
     A.VerticalFlip(p=0.5),
     ToTensorV2(),
     ]
)

test_transform = A.Compose(
    [
    A.Resize(150,150),
    ToTensorV2(),
     ]
)



In [7]:
from torch.utils.data import Dataset, DataLoader

class ImageDataset(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(self.image_paths[idx])
        # image = cv2.imread(image_filepath)


        # print(label)
        if self.transform != False:
            image = self.transform(image=image)["image"]

        return image.float()



train_dataset = ImageDataset(train_list,x_transform)

# test_dataset = ImageDataset(test_list)

In [51]:
# train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# for batch in train_dataloader:
#   print(batch)

In [8]:
import torch
# torch.save(train_dataset, "/content/drive/MyDrive/image_to_str/train_ds.pt")
train_dataset = torch.load("/content/drive/MyDrive/image_to_str/train_ds.pt")

In [9]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 256, (5,5), 1, (1, 1), bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 512, (5, 5), 1, (0, 0), bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 256, (5, 5), 1, (0,0), bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 64, (5,5), 2, (0, 0), bias=True),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 5, (5, 5), 1, (0, 0), bias=True),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [10]:
from torchsummary import summary
print(summary(Generator().to("cuda"), (1,150, 150)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 256, 148, 148]           6,656
       BatchNorm2d-2        [-1, 256, 148, 148]             512
              ReLU-3        [-1, 256, 148, 148]               0
            Conv2d-4        [-1, 512, 144, 144]       3,277,312
       BatchNorm2d-5        [-1, 512, 144, 144]           1,024
              ReLU-6        [-1, 512, 144, 144]               0
            Conv2d-7        [-1, 256, 140, 140]       3,277,056
       BatchNorm2d-8        [-1, 256, 140, 140]             512
              ReLU-9        [-1, 256, 140, 140]               0
           Conv2d-10           [-1, 64, 68, 68]         409,664
      BatchNorm2d-11           [-1, 64, 68, 68]             128
             ReLU-12           [-1, 64, 68, 68]               0
           Conv2d-13            [-1, 5, 64, 64]           8,005
          Sigmoid-14            [-1, 5,

In [26]:
import torch
class Discriminator(nn.Module):
    def __init__(self) -> None:
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 16, (4, 4), (2, 2), (1, 1)),
            nn.LeakyReLU(0.2, True),
            # State size. 64 x 64 x 64
            nn.Conv2d(16, 32, (4, 4), (2, 2), (1, 1)),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2, True),
            # State size. 128 x 32 x 32
            nn.Conv2d(32, 64, (4, 4), (2, 2), (1, 1)),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, True),
            # State size. 256 x 16 x 16
            nn.Conv2d(64, 128, (4, 4), (2, 2), (1, 1)),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, True),
            # State size. 512 x 8 x 8
            nn.Conv2d(128, 256, (4, 4), (2, 2), (1, 1)),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 1, (4, 4), (1, 1), (0, 0)),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.main(x)
        out = torch.flatten(out, 1)

        return out

In [12]:
from torchsummary import summary
print(summary(Discriminator().to("cuda"), (1,150, 150)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 75, 75]             272
         LeakyReLU-2           [-1, 16, 75, 75]               0
            Conv2d-3           [-1, 32, 37, 37]           8,192
       BatchNorm2d-4           [-1, 32, 37, 37]              64
         LeakyReLU-5           [-1, 32, 37, 37]               0
            Conv2d-6           [-1, 64, 18, 18]          32,768
       BatchNorm2d-7           [-1, 64, 18, 18]             128
         LeakyReLU-8           [-1, 64, 18, 18]               0
            Conv2d-9            [-1, 128, 9, 9]         131,072
      BatchNorm2d-10            [-1, 128, 9, 9]             256
        LeakyReLU-11            [-1, 128, 9, 9]               0
           Conv2d-12            [-1, 256, 4, 4]         524,544
      BatchNorm2d-13            [-1, 256, 4, 4]             512
        LeakyReLU-14            [-1, 25

In [31]:
def create_tensor_from_tensor(tensor):
    values = [np.array([[0,0,0],[0,0,0],[0,0,0]]),np.array([[255,0,255],[255,0,255],[255,0,255]]),np.array([[0,255,255],[255,0,255],[0,255,255]]),np.array([[255,255,0],[255,0,255],[255,255,0]]),np.array([[255,255,0],[255,0,255],[255,255,0]]),np.array([[255,255,255],[255,255,255],[255,255,255]])]
    indexes = torch.argmax(tensor,dim=1)
    output_tensor = np.zeros((tensor.shape[0],3*tensor.shape[2],3*tensor.shape[3]))
    for k in range(tensor.shape[0]):
        for i in range(0,tensor.shape[2],1):
            for j in range(0,tensor.shape[3],1):
                index = indexes[k, i, j]
                output_tensor[k,i*3:(i*3)+3, j*3:(j*3)+3] = values[index]
    result = np.zeros((tensor.shape[0],150,150))
    for k in range(tensor.shape[0]):
        result[k,:,:] = cv2.resize(output_tensor[k,:,:], (150,150))
    final_result = np.zeros((tensor.shape[0],1,150,150))
    final_result[:,0,:,:] = result
    return torch.from_numpy(final_result).float()

def process(input_tensor):
    chars  = ['@', '|', '>', '<', ' ']
    tensor = create_tensor_from_tensor(input_tensor)
    return tensor


In [19]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
discriminator = Discriminator().to("cuda")
generator  = Generator().to("cuda")

criterion = nn.BCELoss().to("cuda")
d_optimizer = torch.optim.Adam(discriminator.parameters(), 0.0002, (0.5, 0.999))
g_optimizer = torch.optim.Adam(generator.parameters(),     0.0002, (0.5, 0.999))

In [33]:

from torch.utils.tensorboard import SummaryWriter
import os

writer = SummaryWriter(os.path.join("samples",  "logs", "model_log"))
def train(dataloader, epoch):
    batches = len(dataloader)

    discriminator.train()
    generator.train()

    for index, real in enumerate(train_dataloader):
        # print(real)
        real = real.to("cuda")
        label_size = real.size(0)
        real_label = torch.full([label_size, 1], 1.0, dtype=real.dtype, device="cuda")
        fake_label = torch.full([label_size, 1], 0.0, dtype=real.dtype, device="cuda")
        discriminator.zero_grad()
        # print(real.shape)
        output = discriminator(real)
        d_loss_real = criterion(output, real_label)
        d_loss_real.backward()
        d_real = output.mean().item()
        # Generate a fake image.
        fake = generator(real)
        # print(fake.shape)
        fake = process(fake.to("cpu")).to("cuda")
        # print(fake.shape)
        output = discriminator(fake.to("cuda"))
        d_loss_fake = criterion(output, fake_label)
        d_loss_fake.backward()
        d_fake1 = output.mean().item()
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.step()

        generator.zero_grad()
        output = discriminator(fake)
        g_loss = criterion(output, real_label)
        g_loss.backward()
        g_optimizer.step()
        d_fake2 = output.mean().item()

        iters = index + epoch * batches + 1
        writer.add_scalar("Train_Adversarial/D_Loss", d_loss.item(), iters)
        writer.add_scalar("Train_Adversarial/G_Loss", g_loss.item(), iters)
        writer.add_scalar("Train_Adversarial/D_Real", d_real, iters)
        writer.add_scalar("Train_Adversarial/D_Fake1", d_fake1, iters)
        writer.add_scalar("Train_Adversarial/D_Fake2", d_fake2, iters)
        # Print the loss function every ten iterations and the last iteration in this epoch.
        if (index + 1) % 10 == 0 or (index + 1) == batches:
            print(f"Train stage: adversarial "
                  f"Epoch[{epoch}] "
                  f"D Loss: {d_loss.item():.6f} G Loss: {g_loss.item():.6f} "
                  f"D(Real): {d_real:.6f} D(Fake1)/D(Fake2): {d_fake1:.6f}/{d_fake2:.6f}.")


In [34]:
for epoch in range(5):
    # Train each epoch to generate a model.
    train(train_dataloader, epoch)

# Save the weight of the model under the last Epoch in this stage.
# torch.save(discriminator.state_dict(), os.path.join(exp_dir2, "d-last.pth"))
# torch.save(generator.state_dict(), os.path.join(exp_dir2, "g-last.pth"))

Train stage: adversarial Epoch[0] D Loss: 0.061423 G Loss: 4.044600 D(Real): 0.963671 D(Fake1)/D(Fake2): 0.024123/0.017517.
Train stage: adversarial Epoch[0] D Loss: 0.055247 G Loss: 3.651229 D(Real): 0.996443 D(Fake1)/D(Fake2): 0.050371/0.025959.
Train stage: adversarial Epoch[0] D Loss: 0.076641 G Loss: 4.167836 D(Real): 0.946431 D(Fake1)/D(Fake2): 0.021352/0.015486.
Train stage: adversarial Epoch[0] D Loss: 0.052976 G Loss: 5.704596 D(Real): 0.951695 D(Fake1)/D(Fake2): 0.003459/0.003331.
Train stage: adversarial Epoch[0] D Loss: 0.006374 G Loss: 5.836685 D(Real): 0.996690 D(Fake1)/D(Fake2): 0.003054/0.002919.
Train stage: adversarial Epoch[0] D Loss: 0.014881 G Loss: 5.467773 D(Real): 0.989702 D(Fake1)/D(Fake2): 0.004520/0.004221.
Train stage: adversarial Epoch[0] D Loss: 0.021334 G Loss: 4.460297 D(Real): 0.995016 D(Fake1)/D(Fake2): 0.016205/0.011559.
Train stage: adversarial Epoch[0] D Loss: 0.049204 G Loss: 6.438440 D(Real): 0.953427 D(Fake1)/D(Fake2): 0.001510/0.001599.
Train st

In [35]:
torch.save(generator.to("cpu").state_dict(), "/content/drive/MyDrive/image_to_str/gen_out.pt")